<a href="https://colab.research.google.com/github/keunsooyoon/python202206/blob/main/pythonbasic07_%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 스타벅스 입지 전략 분석

# 거주인구를 기준으로 입지를 선정할까...근무자수를 기준으로 입지를 선정할까...

# 서울시 구별 스타벅스의 숫자 파악



In [1]:
# 코랩에서 셀레니움 크롬 사용 설정

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 8.4 MB/s 
     |████████████████████████████████| 358 kB 51.9 MB/s 
     |████████████████████████████████| 138 kB 56.7 MB/s 
     |████████████████████████████████| 4.0 MB 43.4 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://d

In [3]:
# 크롬에서 별도의 웹 브라우저를 띄울수 없기 때문에 가상화

import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [26]:
driver.get('https://www.starbucks.co.kr/store/store_map.do')


In [27]:
area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element_by_css_selector(area_btn).click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [28]:
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [29]:
all_btn ='#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [30]:
response = driver.page_source
soup = bs(response)
len(soup.select('li.quickResultLstCon'))

# 스타벅스 사이트에서의 서울 지역 매장 개수는 580개가 있지만
# 읽어온 결과는 약간의 차이가 있다. 580 -> 590  현재 위치의 주변에 있는 매장 10개를 한번 더 
# 카운드 하였기 때문에 보이는 차이이다. 따라서 뒤에서 중복 값을 제거한다. 


590

In [31]:
stars = soup.select('li.quickResultLstCon')
star = stars[0]

In [33]:
print(star)

<li class="quickResultLstCon" data-code="3969" data-hlytag="null" data-index="0" data-lat="37.567399" data-long="126.978900" data-name="무교로" data-storecd="1724" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="무교로" data-store="1724" data-yn="N">무교로  </strong> <p class="result_details">서울특별시 중구 무교로 15 (무교동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>


In [46]:
# 위도
lat = star['data-lat']  # 키값 밸류값 형태의 딕셔너리 추출

# 경도
lng = star['data-long']

# 주소
addr = str(star.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]

# 전번
tel = str(star.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

# 지점명
name = star['data-name']



'1522-3232'

In [47]:

# 전체 지점 데이터 프레임 만들기

starbucks_list = []

for star in  stars:
  name = star['data-name']

  lat = star['data-lat']  
  lng = star['data-long']
  
  addr = str(star.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
  tel = str(star.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
  starbucks_list.append([name, lat, lng, addr, tel])




In [48]:
import pandas as pd

df = pd.DataFrame(starbucks_list, columns = ['매장명','위도','경도','주소','전화번호'])
df

,매장명,위도,경도,주소,전화번호
0,무교로,37.567399,126.978900,서울특별시 중구 무교로 15 (무교동),1522-3232
1,한국프레스센터,37.567381,126.977859,서울특별시 중구 세종대로 124 (태평로1가),1522-3232
2,무교동,37.56781712,126.978738,서울특별시 중구 무교로 21 (무교동) 코오롱빌딩 1층,1522-3232
3,시청,37.566290,126.979808,"서울특별시 중구 을지로 19, 삼성화재삼성빌딩 1층 (을지로1가)",1522-3232
4,환구단,37.564448,126.979157,서울특별시 중구 소공로 112 (소공동),1522-3232
...,...,...,...,...,...
585,중랑구청,37.60538908,127.0957558,서울특별시 중랑구 신내로 72,1522-3232
586,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232
587,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
588,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232


In [50]:
# 데이터 프레임에서 중복 항목 제거

df2 = df.drop_duplicates()
df2

,매장명,위도,경도,주소,전화번호
0,무교로,37.567399,126.978900,서울특별시 중구 무교로 15 (무교동),1522-3232
1,한국프레스센터,37.567381,126.977859,서울특별시 중구 세종대로 124 (태평로1가),1522-3232
2,무교동,37.56781712,126.978738,서울특별시 중구 무교로 21 (무교동) 코오롱빌딩 1층,1522-3232
3,시청,37.566290,126.979808,"서울특별시 중구 을지로 19, 삼성화재삼성빌딩 1층 (을지로1가)",1522-3232
4,환구단,37.564448,126.979157,서울특별시 중구 소공로 112 (소공동),1522-3232
...,...,...,...,...,...
585,중랑구청,37.60538908,127.0957558,서울특별시 중랑구 신내로 72,1522-3232
586,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232
587,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
588,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232


In [51]:
# 서울시 구별 스타벅스 매장수 파악

# df2 에 '시군구' 라는 열이름을 생성하여 '주소' 열에서 구만 추출하여 넣자

sgg_names = []             # 구의 이름만 추출하여 담을 리스트 생성 

for addr in df2['주소']:   # 주소를 하나씩 추출
  sgg = addr.split()[1]    # 빈칸으로 분리하여 [1] 을 전부 추출
  sgg_names.append(sgg)    # 구의 이름들만 리스트 담기

df2['시군구명'] = sgg_names # 구 이름 리스트를 데이터 프레임에 그대로 삽입
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,매장명,위도,경도,주소,전화번호,시군구명
0,무교로,37.567399,126.978900,서울특별시 중구 무교로 15 (무교동),1522-3232,중구
1,한국프레스센터,37.567381,126.977859,서울특별시 중구 세종대로 124 (태평로1가),1522-3232,중구
2,무교동,37.56781712,126.978738,서울특별시 중구 무교로 21 (무교동) 코오롱빌딩 1층,1522-3232,중구
3,시청,37.566290,126.979808,"서울특별시 중구 을지로 19, 삼성화재삼성빌딩 1층 (을지로1가)",1522-3232,중구
4,환구단,37.564448,126.979157,서울특별시 중구 소공로 112 (소공동),1522-3232,중구
...,...,...,...,...,...,...
585,중랑구청,37.60538908,127.0957558,서울특별시 중랑구 신내로 72,1522-3232,중랑구
586,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,중랑구
587,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
588,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,중랑구


In [57]:
# 구별 스타벅스의 매장수 파악

# 유정화님 
freq = df2.groupby(['시군구명']).count() 

# 좀 더 보기 좋게 표현
star_count = df2.pivot_table(index = '시군구명',# 구분할 항목
                             values = '매장명', # 계산할 열
                             aggfunc = 'count'
                             ).rename(columns={'매장명':'매장수'})
star_count 

,매장수
시군구명,
강남구,88
강동구,16
강북구,6
강서구,23
관악구,12
광진구,20
구로구,12
금천구,13
노원구,13


In [64]:
# 서울시 구별 인구수, 종사자수 파일 읽어오기

seoul_sgg = pd.read_excel('sgg_pop.xlsx', thousands=',')  # 서울시 구별 인구수, 천단위 구분기호 , 삭제
seoul_biz = pd.read_excel('sgg_biz.xlsx')  # 서울시 구별 종사자수
# star_count                               # 서울시 구별 스타벅스 매장수

seoul_biz

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531
5,동대문구,143858,32994
6,중랑구,99241,27287
7,성북구,113893,24690
8,강북구,69787,18654
9,도봉구,68669,18455


In [65]:
# 세 개의 데이터 프레임을 하나로 병합
# 2개씩 순차적으로 병합한다.

# 먼저 서울시 구별 인구수와 구별 스타벅스 매장수를 병합
seoul_sgg = pd.merge(seoul_sgg, star_count, how='left', on='시군구명')
seoul_sgg

,시군구명,주민등록인구,매장수
0,종로구,177023,39
1,중구,138102,53
2,용산구,250844,23
3,성동구,316113,13
4,광진구,386292,20
5,동대문구,374039,9
6,중랑구,431283,7
7,성북구,484359,14
8,강북구,342669,6
9,도봉구,371095,4


In [66]:
# 다시 근무자수 병합

seoul_sgg = pd.merge(seoul_sgg, seoul_biz, how='left', on='시군구명')
seoul_sgg

,시군구명,주민등록인구,매장수,종사자수,사업체수
0,종로구,177023,39,269106,40871
1,중구,138102,53,423808,66190
2,용산구,250844,23,133446,21178
3,성동구,316113,13,162019,26130
4,광진구,386292,20,123689,24531
5,동대문구,374039,9,143858,32994
6,중랑구,431283,7,99241,27287
7,성북구,484359,14,113893,24690
8,강북구,342669,6,69787,18654
9,도봉구,371095,4,68669,18455


In [67]:
# 주민등록인구 십만명당 매장수

seoul_sgg['거주자_십만명당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['주민등록인구'] / 100000)
seoul_sgg

,시군구명,주민등록인구,매장수,종사자수,사업체수,거주자_십만명당_매장수
0,종로구,177023,39,269106,40871,22.031036
1,중구,138102,53,423808,66190,38.377431
2,용산구,250844,23,133446,21178,9.169045
3,성동구,316113,13,162019,26130,4.112453
4,광진구,386292,20,123689,24531,5.177431
5,동대문구,374039,9,143858,32994,2.406166
6,중랑구,431283,7,99241,27287,1.623064
7,성북구,484359,14,113893,24690,2.890418
8,강북구,342669,6,69787,18654,1.750961
9,도봉구,371095,4,68669,18455,1.077891


In [70]:
# 종사자수 십만명당 매장수

seoul_sgg['종사자_십만명당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['종사자수'] / 100000)
seoul_sgg

,시군구명,주민등록인구,매장수,종사자수,사업체수,거주자_십만명당_매장수,종사자_십만명당_매장수
0,종로구,177023,39,269106,40871,22.031036,14.492430
1,중구,138102,53,423808,66190,38.377431,12.505663
2,용산구,250844,23,133446,21178,9.169045,17.235436
3,성동구,316113,13,162019,26130,4.112453,8.023750
4,광진구,386292,20,123689,24531,5.177431,16.169587
5,동대문구,374039,9,143858,32994,2.406166,6.256169
6,중랑구,431283,7,99241,27287,1.623064,7.053536
7,성북구,484359,14,113893,24690,2.890418,12.292239
8,강북구,342669,6,69787,18654,1.750961,8.597590
9,도봉구,371095,4,68669,18455,1.077891,5.825045


In [71]:
# 업체수 만개당 매장수

seoul_sgg['업체수_만개당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['사업체수'] / 10000)
seoul_sgg

,시군구명,주민등록인구,매장수,종사자수,사업체수,거주자_십만명당_매장수,종사자_십만명당_매장수,업체수_만개당_매장수
0,종로구,177023,39,269106,40871,22.031036,14.492430,9.542218
1,중구,138102,53,423808,66190,38.377431,12.505663,8.007252
2,용산구,250844,23,133446,21178,9.169045,17.235436,10.860327
3,성동구,316113,13,162019,26130,4.112453,8.023750,4.975124
4,광진구,386292,20,123689,24531,5.177431,16.169587,8.152949
5,동대문구,374039,9,143858,32994,2.406166,6.256169,2.727769
6,중랑구,431283,7,99241,27287,1.623064,7.053536,2.565324
7,성북구,484359,14,113893,24690,2.890418,12.292239,5.670312
8,강북구,342669,6,69787,18654,1.750961,8.597590,3.216468
9,도봉구,371095,4,68669,18455,1.077891,5.825045,2.167434
